# Load Packages

In [1]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import torch
from torch.utils.data import DataLoader, random_split
from torch import Generator
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from sklearn.datasets import load_files
from torch import manual_seed as torch_manual_seed
import random
from torch.cuda import max_memory_allocated, set_device, manual_seed_all
from torch.backends import cudnn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import CelebA
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np

# Import Dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
 
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),          # Convert images to tensors
])

cuda


In [3]:
# Load the CelebA dataset -- run as little as possible
train_dataset = CelebA(root='data', split='train', target_type='attr', download=False, transform=transform)
val_dataset = CelebA(root='data', split='valid', target_type='attr', download=False, transform=transform)
test_dataset = CelebA(root='data', split='test', target_type='attr', download=False, transform=transform)

In [4]:
# Create data loaders
train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss = 0

    for batch, (X, y) in enumerate(dataloader):
        # Move to device
        X = X.to(device)
        y = y.to(device)
        y = (y == 1).float()

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        total_loss += loss.item()

        if batch % 20 == 0:
            current = (batch + 1) * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

    # Average loss
    avg_loss = total_loss / num_batches
    return avg_loss


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    total_correct = 0
    total_labels = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            y = (y == 1).float()

            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Apply sigmoid and compute multi-label accuracy
            pred_labels = torch.sigmoid(pred) > 0.5
            correct = (pred_labels == y.bool()).sum().item()
            total_correct += correct
            total_labels += y.numel()

    test_loss /= num_batches
    accuracy = total_correct / total_labels
    print(f"Test Error: \n Accuracy: {100*accuracy:.2f}%, Avg loss: {test_loss:.6f}\n")

# model 1 - efficientNet_B1

In [6]:
dataloaders = {'train' : train_dl, "val" : val_dl}
dataset_sizes = {'train' : len(train_dl.dataset), "val" : len(val_dl.dataset)}

#model itself
model_conv = torchvision.models.efficientnet_b1(weights=torchvision.models.EfficientNet_B1_Weights.DEFAULT)

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier[1] = nn.Linear(num_ftrs, 40)
model_conv = model_conv.to(device)
criterion = nn.BCEWithLogitsLoss()

# Observe that only parameters of final layer are being optimized as opposed to before.
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.1, momentum=0.9)

In [ ]:
#running model
epochs = 1
avg_losses = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model_conv.train()
    LOSS = train_loop(train_dl, model_conv, criterion, optimizer_conv)
    model_conv.eval()
    test_loop(val_dl, model_conv, criterion)
    avg_losses.append(LOSS)

Epoch 1
-------------------------------
loss: 0.200400  [   64/162770]
loss: 0.202975  [ 1344/162770]
loss: 0.195064  [ 2624/162770]
loss: 0.205841  [ 3904/162770]
loss: 0.214517  [ 5184/162770]
loss: 0.201734  [ 6464/162770]
loss: 0.194128  [ 7744/162770]
loss: 0.209708  [ 9024/162770]
loss: 0.214482  [10304/162770]
loss: 0.183043  [11584/162770]
loss: 0.199749  [12864/162770]
loss: 0.201965  [14144/162770]
loss: 0.205093  [15424/162770]
loss: 0.187492  [16704/162770]
loss: 0.217952  [17984/162770]
loss: 0.207203  [19264/162770]
loss: 0.201325  [20544/162770]
loss: 0.200443  [21824/162770]
loss: 0.209946  [23104/162770]
loss: 0.187947  [24384/162770]
loss: 0.208104  [25664/162770]
loss: 0.200787  [26944/162770]
loss: 0.213014  [28224/162770]
loss: 0.216259  [29504/162770]
loss: 0.198329  [30784/162770]
loss: 0.207416  [32064/162770]
loss: 0.209578  [33344/162770]
loss: 0.198900  [34624/162770]
loss: 0.192838  [35904/162770]
loss: 0.206435  [37184/162770]
loss: 0.217654  [38464/162770]

In [ ]:
# plot the avg loss for model 1
cpu_avg_losses = [loss.cpu().item() for loss in avg_losses]
plt.plot(cpu_avg_losses)
plt.xlabel("Epoch Number")
plt.ylabel("Average loss for Model 1 - efficientNet_b1")
plt.title("Avergae loss for Model 1 per epoch")
plt.show()